# FeatherFace Baseline Training and Evaluation

This notebook reproduces the original FeatherFace training process following the author's instructions.

## Overview
- Model: FeatherFace with MobileNetV1 0.25x backbone
- Dataset: WIDERFace (auto-download)
- Expected Results: 0.49M parameters, 90.8% mAP
- Uses original training scripts for faithful reproduction

## 1. Installation and Environment Setup

In [ ]:
# Install required packages
!pip install -e .
!pip install gdown requests

In [ ]:
# Verify imports and check GPU
import torch
import torchvision
import cv2
import numpy as np
import os
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import gdown
import requests
import zipfile
import tarfile
import json
from datetime import datetime

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## 2. Automatic Dataset Download - WIDERFace

Download and organize the WIDERFace dataset automatically.

In [ ]:
# Create data directories
data_root = Path('./data/widerface')
data_root.mkdir(parents=True, exist_ok=True)

# WIDERFace download links
WIDERFACE_GDRIVE_ID = '11UGV3nbVv1x9IC--_tK3Uxf7hA6rlbsS'
WIDERFACE_URL = f'https://drive.google.com/uc?id={WIDERFACE_GDRIVE_ID}'

def download_widerface():
    """Download WIDERFace dataset from Google Drive"""
    output_path = data_root / 'widerface.zip'
    
    if not output_path.exists():
        print("Downloading WIDERFace dataset...")
        print("This may take several minutes depending on your connection.")
        
        try:
            gdown.download(WIDERFACE_URL, str(output_path), quiet=False)
            print(f"✓ Downloaded to {output_path}")
        except Exception as e:
            print(f"❌ Download failed: {e}")
            print("Please download manually from:")
            print(f"  {WIDERFACE_URL}")
            return False
    else:
        print(f"✓ Dataset already downloaded: {output_path}")
    
    return True

# Download dataset
if download_widerface():
    print("\n✅ Dataset download complete!")
else:
    print("\n❌ Please download the dataset manually.")

In [ ]:
# Extract dataset
def extract_widerface():
    """Extract WIDERFace dataset"""
    zip_path = data_root / 'widerface.zip'
    
    if not zip_path.exists():
        print("❌ Dataset zip file not found. Please download first.")
        return False
    
    # Check if already extracted
    if (data_root / 'train' / 'label.txt').exists() and \
       (data_root / 'val' / 'wider_val.txt').exists():
        print("✓ Dataset already extracted")
        return True
    
    print("Extracting dataset...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_root)
        print("✓ Dataset extracted successfully")
        return True
    except Exception as e:
        print(f"❌ Extraction failed: {e}")
        return False

# Extract dataset
if extract_widerface():
    print("\n✅ Dataset ready for use!")
else:
    print("\n❌ Please extract the dataset manually.")

In [ ]:
# Verify dataset structure
def verify_dataset():
    """Verify WIDERFace dataset structure"""
    required_files = [
        data_root / 'train' / 'label.txt',
        data_root / 'val' / 'wider_val.txt'
    ]
    
    all_present = True
    for file_path in required_files:
        if file_path.exists():
            print(f"✓ Found: {file_path}")
        else:
            print(f"✗ Missing: {file_path}")
            all_present = False
    
    # Check for images
    for split in ['train', 'val']:
        img_dir = data_root / split / 'images'
        if img_dir.exists():
            img_count = len(list(img_dir.glob('**/*.jpg')))
            print(f"✓ {split} images: {img_count} found")
        else:
            print(f"✗ {split} images directory not found")
            all_present = False
    
    return all_present

dataset_ready = verify_dataset()
print(f"\nDataset verification: {'PASSED ✅' if dataset_ready else 'FAILED ❌'}")

## 3. Automatic Pre-trained Weights Download

In [ ]:
# Create weights directory
weights_dir = Path('./weights')
weights_dir.mkdir(exist_ok=True)

# Pre-trained weights info
PRETRAIN_GDRIVE_ID = '1oZRSG0ZegbVkVwUd8wUIQx8W7yfZ_ki1'
PRETRAIN_URL = f'https://drive.google.com/uc?id={PRETRAIN_GDRIVE_ID}'
PRETRAIN_FILENAME = 'mobilenetV1X0.25_pretrain.tar'

def download_pretrained_weights():
    """Download pre-trained MobileNetV1 0.25x weights"""
    output_path = weights_dir / PRETRAIN_FILENAME
    
    if not output_path.exists():
        print("Downloading pre-trained weights...")
        try:
            gdown.download(PRETRAIN_URL, str(output_path), quiet=False)
            print(f"✓ Downloaded to {output_path}")
        except Exception as e:
            print(f"❌ Download failed: {e}")
            print("Please download manually from:")
            print(f"  {PRETRAIN_URL}")
            return False
    else:
        print(f"✓ Pre-trained weights already exist: {output_path}")
    
    return True

# Download weights
if download_pretrained_weights():
    print("\n✅ Pre-trained weights ready!")
else:
    print("\n❌ Please download weights manually.")


## 4. Configuration Check

Following the author's instructions to check network configuration in `data/config.py`

In [ ]:
# Import configuration
from data import cfg_mnet

# Display configuration as mentioned in README
print("=== FeatherFace Configuration (cfg_mnet) ===")
print(f"Network name: {cfg_mnet['name']}")
print(f"Input image size: {cfg_mnet['image_size']}")
print(f"Batch size: {cfg_mnet['batch_size']}")
print(f"Learning rate: {cfg_mnet['lr']}")
print(f"Training epochs: {cfg_mnet['epoch']}")
print(f"GPU train: {cfg_mnet['gpu_train']}")
print(f"\nMin sizes: {cfg_mnet['min_sizes']}")
print(f"Steps: {cfg_mnet['steps']}")
print(f"Variance: {cfg_mnet['variance']}")
print(f"Clip: {cfg_mnet['clip']}")
print(f"Loc weight: {cfg_mnet['loc_weight']}")
print(f"\nNote: You can modify these in data/config.py before training")

## 5. Training Using Original Scripts

Following the author's command: `CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 train.py --network mobile0.25`

In [ ]:
# Option 1: Train using the author's script (recommended for full training)
print("To train the model, run this command in terminal:")
print("\nCUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 train.py --network mobile0.25")
print("\nOr run the cell below to execute from notebook:")

In [ ]:
# Run training script from notebook
# Uncomment to run full training (will take several hours)

# import subprocess
# import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# cmd = ['torchrun', '--standalone', '--nproc_per_node=1', '../train.py', '--network', 'mobile0.25']
# subprocess.run(cmd)

## 6. Quick Training Demo (Alternative)

For demonstration purposes, here's a simplified training loop:

In [ ]:
# Import necessary modules for demo
from models.retinaface import RetinaFace
from layers.modules import MultiBoxLoss
from data import WiderFaceDetection, detection_collate
import data.data_augment as data_augment
from torch.utils.data import DataLoader

# Model setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Create model
net = RetinaFace(cfg=cfg_mnet)
print(f"Model created: {cfg_mnet['name']}")

# Count parameters
total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,} ({total_params/1e6:.2f}M)")

# Load pre-trained backbone
pretrain_path = weights_dir / PRETRAIN_FILENAME
if pretrain_path.exists():
    print(f"\nLoading pre-trained weights from {pretrain_path}")
    state_dict = torch.load(str(pretrain_path), map_location=device)
    # Load only matching keys
    model_dict = net.state_dict()
    pretrained_dict = {k: v for k, v in state_dict.items() if k in model_dict}
    model_dict.update(pretrained_dict)
    net.load_state_dict(model_dict)
    print(f"Loaded {len(pretrained_dict)} pre-trained layers")

net = net.to(device)

In [ ]:
# Quick training demo (2 epochs only)
def train_demo(net, num_epochs=2):
    """Demo training for notebook - use train.py for full training"""
    # Data setup
    rgb_mean = (104, 117, 123)
    train_dataset = WiderFaceDetection(
        str(data_root / 'train/label.txt'),
        data_augment.preproc(cfg_mnet['image_size'], rgb_mean)
    )
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=cfg_mnet['batch_size'],
        shuffle=True,
        num_workers=2,
        collate_fn=detection_collate,
        pin_memory=True
    )
    
    # Loss and optimizer
    criterion = MultiBoxLoss(cfg_mnet['num_classes'], 0.35, True, 0, True, 7, 0.35, False)
    optimizer = torch.optim.SGD(net.parameters(), lr=cfg_mnet['lr'], momentum=0.9, weight_decay=5e-4)
    
    print(f"Training samples: {len(train_dataset)}")
    print(f"Training batches: {len(train_loader)}")
    print(f"\nStarting demo training for {num_epochs} epochs...")
    
    net.train()
    for epoch in range(num_epochs):
        for batch_idx, (images, targets) in enumerate(train_loader):
            if batch_idx >= 5:  # Only 5 batches for demo
                break
                
            images = images.to(device)
            targets = [anno.to(device) for anno in targets]
            
            out = net(images)
            loss_l, loss_c, loss_landm = criterion(out, targets)
            loss = cfg_mnet['loc_weight'] * loss_l + loss_c + loss_landm
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if batch_idx % 2 == 0:
                print(f"Epoch [{epoch+1}/{num_epochs}] Batch [{batch_idx}/{min(5, len(train_loader))}] "
                      f"Loss: {loss.item():.4f}")
    
    print("\n✅ Demo training complete!")
    print("Note: For full training, use train.py script as shown above.")

# Uncomment to run demo
# train_demo(net, num_epochs=2)

## 7. Evaluation Using Original Scripts

Following the author's evaluation process:
1. Generate txt files using `test_widerface.py`
2. Evaluate using WIDERFace evaluation tools

In [ ]:
# Step 1: Generate detection results
print("=== Evaluation Step 1: Generate Detection Results ===")
print("\nRun this command after training completes:")
print("python test_widerface.py --trained_model ./weights/mobilenet0.25_Final.pth --network mobile0.25 --origin_size True")
print("\nThis will generate detection results in ./widerface_txt/")

In [ ]:
# Step 2: Evaluate results
print("=== Evaluation Step 2: Calculate mAP ===")
print("\nRun these commands after generating detection results:")
print("cd ./widerface_evaluate")
print("python evaluation.py -p ./widerface_txt -g ./eval_tools/ground_truth")
print("\nThis will output the mAP scores for Easy, Medium, and Hard subsets.")

## 8. Test on Single Image

In [ ]:
# Simple test function for single image
def test_single_image(image_path, model_path=None):
    """Test detection on a single image"""
    # Load model if path provided
    if model_path and Path(model_path).exists():
        net.load_state_dict(torch.load(model_path, map_location=device))
        print(f"Loaded model from {model_path}")
    
    # Run detection using detect.py logic
    from detect import detect_single_image
    
    # This would use the author's detect.py script
    # For now, we'll use a simplified version
    print(f"\nTo test on image '{image_path}', run:")
    print(f"python detect.py -m {model_path or './weights/mobilenet0.25_Final.pth'} --image_path {image_path}")

print("Detection functions ready.")
print("Place test images in the notebook directory and use test_single_image('image.jpg')")

## 9. Export to ONNX

In [ ]:
# Export model to ONNX
def export_onnx(model, save_path='./weights/featherface_baseline.onnx', input_size=640):
    """Export model to ONNX format"""
    model.eval()
    
    # Create dummy input
    dummy_input = torch.randn(1, 3, input_size, input_size).to(device)
    
    print(f"Exporting to {save_path}...")
    
    # Export
    torch.onnx.export(
        model,
        dummy_input,
        save_path,
        export_params=True,
        opset_version=11,
        do_constant_folding=True,
        input_names=['input'],
        output_names=['bbox', 'confidence', 'landmarks'],
        dynamic_axes={
            'input': {0: 'batch_size'},
            'bbox': {0: 'batch_size'},
            'confidence': {0: 'batch_size'},
            'landmarks': {0: 'batch_size'}
        }
    )
    
    print(f"✓ Model exported to {save_path}")
    print(f"  Size: {os.path.getsize(save_path) / 1e6:.2f} MB")
    
    # Verify ONNX model
    import onnx
    onnx_model = onnx.load(save_path)
    onnx.checker.check_model(onnx_model)
    print("✓ ONNX model is valid")

# Uncomment to export
# export_onnx(net)

## 10. Summary and Results

### Expected Baseline Results (from paper)
- Model: FeatherFace with MobileNetV1 0.25x
- Parameters: 0.49M
- Performance on WIDERFace:
  - Easy: 90.8%
  - Medium: 88.1%
  - Hard: 73.8%

### Training Commands Summary
```bash
# Training
CUDA_VISIBLE_DEVICES=0 torchrun --standalone --nproc_per_node=1 train.py --network mobile0.25

# Testing
python test_widerface.py --trained_model ./weights/mobilenet0.25_Final.pth --network mobile0.25 --origin_size True

# Evaluation
cd ./widerface_evaluate
python evaluation.py -p ./widerface_txt -g ./eval_tools/ground_truth
```

### Next Steps
1. Complete full training (250 epochs)
2. Evaluate on WIDERFace test set
3. Proceed to Phase 2: FeatherFace V2 optimizations

In [ ]:
# Save notebook execution summary
summary = {
    'notebook': '01_train_evaluate_featherface.ipynb',
    'model': 'FeatherFace Baseline',
    'backbone': 'MobileNetV1 0.25x',
    'parameters': f"{total_params/1e6:.2f}M" if 'total_params' in locals() else "0.49M",
    'dataset': 'WIDERFace',
    'config': cfg_mnet['name'],
    'expected_map': {'easy': 90.8, 'medium': 88.1, 'hard': 73.8},
    'timestamp': datetime.now().isoformat()
}

# Save summary
import json
os.makedirs('./results', exist_ok=True)
with open('./results/baseline_summary.json', 'w') as f:
    json.dump(summary, f, indent=4)

print("✅ Baseline notebook execution complete!")
print(json.dumps(summary, indent=2))